In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = "https://pokemondb.net/pokedex/all"

# Fetch the page content
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing Pokémon data
table = soup.find('table', {'id': 'pokedex'})

# Extract the table headers and rows
headers = [th.text for th in table.find('thead').find_all('th')]
rows = table.find('tbody').find_all('tr')

# Extract the data
pokemon_data = []
for row in rows:
    columns = row.find_all('td')
    sprite_url = row.find('td').find('span').find('img')['src']  # Fetch sprite image URL
    data = [col.text.strip() for col in columns]
    data.insert(1, sprite_url)  # Insert sprite URL after the Pokémon ID
    pokemon_data.append(data)

# Add sprite header to the list of headers
headers.insert(1, 'Sprite URL')

# Create a pandas DataFrame
df = pd.DataFrame(pokemon_data, columns=headers)

# Display the first few rows of the DataFrame
print(df.head())
